# Load Data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Input,Dropout, LSTM
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings("ignore")
sns.set()

In [ ]:
traindf = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv')
testdf = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv')
traindf.info()

# Preprocessing & Analyzing

In [ ]:
traindf["Close"] =traindf["Close"].apply(lambda x: float(x.strip("\"").replace(',','')))
traindf["Volume"] =traindf["Volume"].apply(lambda x: float(x.strip("\"").replace(',','')))
traindf.info()

In [ ]:
train_x=traindf[["Open","High","Low","Close","Volume"]]

In [ ]:
fig,ax = plt.subplots(3,2,figsize=(17,15))
sns.lineplot(ax=ax[0][0],x=traindf["Date"],y=traindf["Open"])
sns.lineplot(ax=ax[0][1],x=traindf["Date"],y=traindf["High"])
sns.lineplot(ax=ax[1][0],x=traindf["Date"],y=traindf["Low"])
sns.lineplot(ax=ax[1][1],x=traindf["Date"],y=traindf["Close"])
sns.lineplot(ax=ax[2][0],x=traindf["Date"],y=traindf["Volume"])
#g.set_xticklabels(g.get_xticklabels(),rotation=90)
plt.subplot(3,2,6)
plt.axis("off")
plt.show()

In [ ]:
scaler = StandardScaler()
train_x = scaler.fit_transform(traindf[["Open","High","Low","Close","Volume"]])

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca_res = pca.fit_transform(train_x)

In [ ]:
pca_res

# Split and arrange Data

In [ ]:
train_size = int(len(traindf) * 0.65)
x_train = pca_res[0:train_size,:]
x_test = pca_res[train_size:,:]

In [ ]:
#thanks to https://www.kaggle.com/kshashank03/stock-prediction-using-tensorflow-time-series
def create_dataset(data, time_step=1):
    dataX, dataY = [], []
    for i in range(len(data) - time_step - 1):
        dataX.append(data[i:(i + time_step), :])
        dataY.append(data[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

time_step = 12
X_train, y_train = create_dataset(x_train, time_step)
X_test, y_test = create_dataset(x_test, time_step)

# shape 3-D(sample, timesteps, features)

# Build Deep learning Model

In [ ]:
#CNN Model Definition
def create_cnnmodel():
    # define the model
    model = Sequential()
    model.add(Conv1D(filters=3,kernel_size=2,strides=1, activation="relu"))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#LSTM Model Definition
def create_lstmmodel():
    # define the model
    model = Sequential()
    model.add(LSTM(3))
    model.add(Dense(3,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#LSTM in GridSearch
model = KerasClassifier(build_fn=create_lstmmodel, verbose=0)
batch_size = [5, 7,10]
epochs = [1,2,3,5]
param_grid = dict(batch_size=batch_size, epochs=epochs)
lstm_grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
lstm_result = lstm_grid.fit(X_train,y_train)
# summarize results
print("Best: %f using %s" % (lstm_result.best_score_, lstm_result.best_params_))

In [ ]:
#CNN in GridSearch
model = KerasClassifier(build_fn=create_cnnmodel, verbose=0)
batch_size = [5, 10]
epochs = [1, 2, 3]
param_grid = dict(batch_size=batch_size, epochs=epochs)
cnn_grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
cnn_result = cnn_grid.fit(X_train,y_train)
# summarize results
print("Best: %f using %s" % (cnn_result.best_score_, cnn_result.best_params_))

# Evaluate Model

In [ ]:
lstm_result.score(X_test,y_test)

In [ ]:
cnn_result.score(X_test,y_test)